# Assignment 1

## 1.0 Import and install python libraries

In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
import summarytools
from summarytools import dfSummary
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 3.0 Load data 

In [2]:
X_train = pd.read_csv("./data/Rscores_X_train.csv")
X_test = pd.read_csv("./data/Rscores_X_test.csv")
y_train = pd.read_csv("./data/Rscores_y_train.csv")
y_test = pd.read_csv("./data/Rscores_y_test.csv")

In [3]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# Logistic Regression

In [4]:
log_reg_model = LogisticRegression(penalty='none',max_iter=2000)
_ = log_reg_model.fit(X_train, np.ravel(y_train,order='C'))

c:\Users\mukes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846


# Randome search - Logistic

In [6]:
#random search in log
score_measure = "precision_macro"
kfolds = 15

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

logreg_rand = LogisticRegression()

rand_search_log = RandomizedSearchCV(estimator = logreg_rand, param_distributions=param_grid, cv=kfolds, n_iter=4,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_log.fit(X_train,np.ravel(y_train,order='C'))

print(f"The best {score_measure} score is {rand_search_log.best_score_}")
print(f"... with parameters: {rand_search_log.best_params_}")

bestRecallTree = rand_search_log.best_estimator_

Fitting 15 folds for each of 4 candidates, totalling 60 fits
The best precision_macro score is 0.3455509705431804
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}


In [7]:
model_preds = rand_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_search_log_l2", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762


# Grid search - Logistic

In [8]:
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1'],
    'solver': ['liblinear']
}

logreg_grid = LogisticRegression()
grid_search_log = GridSearchCV(logreg_grid, param_grid=param_grid, cv=kfolds, 
                               scoring=score_measure, verbose=1, n_jobs=-1, 
                               return_train_score=True, error_score='raise')

_ = grid_search_log.fit(X_train, np.ravel(y_train))

print(f"The best macro-averaged precision score is {grid_search_log.best_score_}")
print(f"... with parameters: {grid_search_log.best_params_}")

bestLogReg = grid_search_log.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best macro-averaged precision score is 0.36766437367434
... with parameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [9]:
model_preds = grid_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_search_log_l1", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762


# SVM

In [10]:
#svm with linear kernel
svm_lin_model = SVC(kernel="linear",probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [11]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"lin svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640


# SVM with random search

In [12]:
#random search in SVM
from scipy.stats import uniform
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'kernel':['linear'],
    'C': uniform(loc=0, scale=4),
    'gamma': uniform(loc=0, scale=1)
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=4,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train,np.ravel(y_train))

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best precision_macro score is 0.4075079575468171
... with parameters: {'C': 3.730229437354635, 'gamma': 0.12812444792935673, 'kernel': 'linear'}


In [13]:
model_preds = rand_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640


# SVM Grid search

In [82]:
#grid search in SVM
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train,np.ravel(y_train))

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
model_preds = grid_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

# Dtree

In [14]:
dtree=DecisionTreeClassifier()
_ = dtree.fit(X_train, y_train)

In [15]:
model_preds = dtree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516


# DTree Random search

In [16]:
#random with tree
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,42),  
    'min_samples_leaf': np.arange(2,42),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(2,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search_tree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
 
_ = rand_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_tree.best_score_}")
print(f"... with parameters: {rand_search_tree.best_params_}")

bestRecallTree = rand_search_tree.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best precision_macro score is 0.9737313755412134
... with parameters: {'min_samples_split': 4, 'min_samples_leaf': 3, 'max_leaf_nodes': 34, 'max_depth': 36, 'criterion': 'entropy'}


In [17]:
model_preds = rand_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491


# Dtree with Grid

In [18]:
#grid with tree
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_tree.best_score_}")
print(f"... with parameters: {grid_search_tree.best_params_}")

bestRecallTree = grid_search_tree.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best precision_macro score is 0.9753981515046645
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 37, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 3, 'min_samples_split': 3}


In [19]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491


## MLP

In [20]:
from __future__ import print_function
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets

In [21]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40), activation='relu',solver='adam', max_iter=200)
_ = ann.fit(X_train, np.ravel(y_train))

Wall time: 12 s


In [22]:
%%time
y_pred = ann.predict(X_test)

Wall time: 15.9 ms


In [23]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"NN-60,50,40", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491


## Deep Network

In [24]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [25]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(14))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(400, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [26]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [27]:
# Fit the model

history = model.fit(X_train, np.ravel(y_train), 
                    validation_data=(X_test, np.ravel(y_test)), 
                    epochs=100, batch_size=100)

Epoch 1/100
7/7 [==============================] - 1s 54ms/step - loss: 40.5125 - accuracy: 0.3426 - val_loss: 9.2056 - val_accuracy: 0.4932
Epoch 2/100
7/7 [==============================] - 0s 11ms/step - loss: 4.2070 - accuracy: 0.3735 - val_loss: 1.2963 - val_accuracy: 0.4589
Epoch 3/100
7/7 [==============================] - 0s 10ms/step - loss: 1.2158 - accuracy: 0.4162 - val_loss: 1.0609 - val_accuracy: 0.4932
Epoch 4/100
7/7 [==============================] - 0s 10ms/step - loss: 1.1086 - accuracy: 0.4574 - val_loss: 1.1202 - val_accuracy: 0.4281
Epoch 5/100
7/7 [==============================] - 0s 10ms/step - loss: 1.0587 - accuracy: 0.4765 - val_loss: 1.1191 - val_accuracy: 0.3801
Epoch 6/100
7/7 [==============================] - 0s 11ms/step - loss: 1.0593 - accuracy: 0.4588 - val_loss: 1.0836 - val_accuracy: 0.4281
Epoch 7/100
7/7 [==============================] - 0s 9ms/step - loss: 1.0384 - accuracy: 0.4441 - val_loss: 0.9970 - val_accuracy: 0.5068
Epoch 8/100
7/7 [===

In [28]:
# evaluate the model

scores = model.evaluate(X_test, np.ravel(y_test), verbose=0)
scores
# In results, first is loss, second is accuracy

[0.9682949781417847, 0.4931506812572479]

In [29]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.97
accuracy: 49.32%


## random - uniform

In [30]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=14)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann

Wall time: 34.4 ms


In [31]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=120,
    dropout = 0.0
)

In [64]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.05],
    'model__hidden_layer_sizes': [(70,)],
    'model__dropout': [0, 0.1],
    'batch_size':[10],
    'epochs':[10],
    'optimizer':[adam]
}
keras_clf.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [65]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

...

c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 991us/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

In [66]:
random_DNN_uniform = rnd_search_cv.best_estimator_

In [67]:
model_preds = random_DNN_uniform.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"random-DNN-uniform-e10-hls-150", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

30/30 [==============================] - 0s 980us/step


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,random-DNN-uniform-e10-hls-120,0.653543,0.812500,0.750000,0.780000


## random normal

In [43]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=14)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotNormal(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann

Wall time: 0 ns


In [68]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=120,
    dropout = 0.0
)

In [73]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.05],
    'model__hidden_layer_sizes': [(70,)],
    'model__dropout': [0, 0.1],
    'batch_size':[10],
    'epochs':[10],
    'optimizer':[adam]
}
keras_clf.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [74]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

...

c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 945us/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

14/14 [==============================] - 0s 1ms/step


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\mukes\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\mukes\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

In [75]:
rand_DNN_normal = rnd_search_cv.best_estimator_

In [76]:
model_preds = rand_DNN_normal.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"random-DNN-normal-e10-hls120", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

30/30 [==============================] - 0s 918us/step


In [49]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,random-DNN-uniform-e10-hls-120,0.653543,0.812500,0.750000,0.780000


In [77]:
performance.sort_values(by='Precision')

,model,Accuracy,Precision,Recall,F1
0,random-DNN-normal-e10-hls70,0.614754,0.788889,0.717172,0.751323
0,random-DNN-uniform-e10-hls-120,0.653543,0.812500,0.750000,0.780000
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,random-DNN-normal-e10-hls120,0.716535,0.841584,0.809524,0.825243
0,random-DNN-uniform-e10-hls-150,0.716418,0.846154,0.800000,0.822430
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491


In [78]:
performance.sort_values(by='Accuracy')

,model,Accuracy,Precision,Recall,F1
0,random-DNN-normal-e10-hls70,0.614754,0.788889,0.717172,0.751323
0,random-DNN-uniform-e10-hls-120,0.653543,0.812500,0.750000,0.780000
0,random-DNN-uniform-e10-hls-150,0.716418,0.846154,0.800000,0.822430
0,random-DNN-normal-e10-hls120,0.716535,0.841584,0.809524,0.825243
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,default logistic,0.827586,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,rand_poly_tree,0.994565,0.993007,1.000000,0.996491


Here we can say that when accuracy scores are compared with different models dtree is the best model among them and also the precision is more for dtree model. I have observed that when the data set is trained with Deep nueral network, changes in epoch and hidden layer size had a significant impact in both cases of uniform and normal distribution of weights. Increasing the hidden layer size has led to increase in the accuracy and little comparable change in precision. When compared MLP and Keras model with other models Dtree is best performing model in all kinds of metrics.